In [1]:
from fastai import *
from fastai.text import *

In [15]:
path = untar_data(URLs.IMDB,dest='data')

In [102]:
path.ls()

[PosixPath('data/imdb/test'),
 PosixPath('data/imdb/README'),
 PosixPath('data/imdb/unsup'),
 PosixPath('data/imdb/imdb.vocab'),
 PosixPath('data/imdb/tmp_lm'),
 PosixPath('data/imdb/train'),
 PosixPath('data/imdb/tmp_clas')]

In [17]:
data_lm = TextLMDataBunch.from_folder(path,test='test')

In [18]:
data_lm.show_batch()

idx,text
0,"new . xxmaj have we all forgotten xxmaj de xxmaj sica 's "" xxmaj bicycle xxmaj thief "" ? xxmaj or anything by xxmaj hitchcock ? \n \n xxmaj so all we get is the extended metaphor of the juvenile as xxmaj truffaut , who spends all his free time ' screwing up his eyes ' at the movies , who wrecks schoolroom discipline , gets accused of xxunk"
1,"film and plays his usual , smarmy , egotistical , snotty character that was actually endearing in "" xxmaj pretty xxmaj in xxmaj pink "" and has xxup not been amusing ever since . xxmaj grating does n't even begin to describe his performance . xxmaj ricky ( xxmaj mark xxmaj xxunk ) is a muscular , blonde , struggling actor who ( gasp ! ) is only worried about"
2,"xxmaj clan , they 're nasty & they do bad things but they ai n't go no soul . i see a lot of reviews from people that liked this , and i guess i do n't know what i missed , but i found it to be very mediocre & i would n't recommend it to anyone , really . 4 out of 10 . xxbos xxmaj this movie"
3,"see if you can count the cliches . xxmaj and yes , xxmaj blair gets possessed , as if you did n't see xxup that coming down xxmaj main xxmaj street followed by a marching band . \n \n xxmaj no stars . "" xxmaj witchery "" - these witches will give you xxunk . xxbos xxmaj charlotte 's deadly beauty and lethal kicks make the movie cooler than"
4,"stallone , who co - wrote the script with xxmaj james xxmaj cameron ( a long way from the exciting xxup terminator made the previous xxunk seems to have given the xxmaj rambo character as little to say in understandable xxmaj english , and merely comes out with moronic grunts , almost as though he has invented his own brand of patois only understandable to himself . xxmaj maybe his"


In [10]:
import spacy
import random
import nltk 
from nltk.corpus import wordnet 
import en_core_web_lg
import re

In [106]:
def synalter_Noun_Verb(cand_word,POS):
    thresh = -1
    scores = []
    synonyms = wordnet.synsets(cand_word)
    for w2 in synonyms:
        try:
            w1 = wordnet.synset(cand_word+'.'+POS+'.01') 
        #w2 = wordnet.synset(word+'.'+POS+'.01') # n denotes noun 
            if (w1.wup_similarity(w2)>thresh):
                scores.append(w1.wup_similarity(w2))
            return synonyms[np.argmax(scores)].name().split('.')[0].replace('_',' ')
        except:
            if not scores:
                for synset in synonyms:
                    word=synset.name().split('.')[0].replace('_',' ')
                    #print(word)
                    token = nlp(word)
                    token_main = nlp(cand_word)
                    if token.vector_norm and token_main.vector_norm and float(token.similarity(token_main)) > thresh:
                        scores.append(float(token.similarity(token_main)))
            return synonyms[np.argmax(scores)].name().split('.')[0].replace('_',' ')

In [20]:
nlp = spacy.load('en_core_web_lg')

In [6]:
doc = nlp('bright car running')
[tok.pos_ for tok in doc]

['ADJ', 'NOUN', 'VERB']

In [105]:
def change_words(word_str,pos,output_text):
    change_word=synalter_Noun_Verb(word_str,pos)
    if change_word:
        search_word = re.search(r'\b('+word_str+r')\b', output_text)
        Loc = search_word.start()
        output_text = output_text[:int(Loc)] + change_word + output_text[int(Loc) + len(word_str):] 
        
    return output_text

In [100]:
from glob import glob

In [140]:
files = glob(str(path)+'/**/*.txt',recursive=True)
len(files)

100006

In [141]:
files = [f for f in files if '_aug.txt' not in f] #skip already augmented files
len(files)

100002

In [144]:
class text_augmenter(object):
#    def __init__(self):
    
    def __call__(self, fn,i): 
        #print(f'augmenting file {fn}')
        output_text = open(fn,'r').read()
        #print("Sentence: "+text)
        counts = Counter(output_text.split())
        noun = []
        verb = []
        doc = nlp(' '.join(w for w in [key for key,value in counts.items() if value ==1]))
        noun = [tok.text for tok in doc if tok.pos_ == 'NOUN']
        verb = [tok.text for tok in doc if tok.pos_ == 'VERB']
        adj = [tok.text for tok in doc if tok.pos_ == 'ADJ']
        cand_words = noun+verb+adj
        len_all = len(verb+noun+adj)
        random.seed(4)
        temp = random.sample(range(len_all), random.randint(0,int(len_all*0.7)))
        #print([cand_words[i] for i in temp])
        for i in temp:
            word_str = cand_words[i]
            if i<len(verb):output_text = change_words(word_str,'v',output_text)
            elif i<len(noun): output_text = change_words(word_str,'n',output_text)
            else:output_text=change_words(word_str,'a',output_text)
        new_fn = fn.replace('.txt','_aug.txt')
        with open(new_fn,'w') as f:
            f.write(output_text)
        #print(f'saving augmented file to {new_fn}')

In [ ]:
parallel(text_augmenter(),files)

In [146]:
files = glob(str(path)+'/**/*.txt',recursive=True)
len(files)

100336